In [2]:
import os 
import sys 
import pandas as pd 
import numpy as np
from collections import Counter
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import datasets
from sklearn import tree
from six import StringIO
from IPython.display import Image 
import pydot
import pydotplus
sys.path.append('/Users/yantingting/PycharmProjects/modeling/')
import summary_statistics as ss
import metrics as mt 
import feature_selection as fs 
from feature_selection import *
import misc_utils as mu
from data_io_utils import *
import data_encryption as de
import plot_tools as pt 
import data_processing as dp 
import database_conncet as dc 
# 必须要在该界面设置中文的参数
import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.rcParams['font.sans-serif'] = ['STFangsong']

fs_obj = fs.FeatureSelection()
pf = mt.Performance()
bw = mt.BinWoe()

In [3]:
file_path = '/Users/yantingting/Seafile/风控/模型/10 印尼/202004 老客决策树/01 Data'
# file_path_result ='/Users/yantingting/Seafile/风控/模型/10 印尼/202004 老客决策树/02 Result'

In [7]:
all_var.columns.tolist()

['loan_id',
 'effective_date',
 'due_date',
 'approved_period',
 'late_day',
 'dpd1',
 'age',
 'monthly_salary',
 'device_approval_ANDROID',
 'device_approval_IOS',
 'device_approval_API',
 'product_id_1',
 'product_id_2',
 'gender_female',
 'gender_male',
 'province_JAWA BARAT',
 'province_DKI JAKARTA',
 'province_JAWA TIMUR',
 'province_JAWA TENGAH',
 'province_BANTEN',
 'province_SUMATERA UTARA',
 'province_SULAWESI SELATAN',
 'province_SUMATERA SELATAN',
 'province_BALI',
 'province_KALIMANTAN TIMUR',
 'province_DAERAH ISTIMEWA YOGYAKARTA',
 'province_LAMPUNG',
 'province_SULAWESI UTARA',
 'province_RIAU',
 'province_SUMATERA BARAT',
 'province_KEPULAUAN RIAU',
 'province_KALIMANTAN BARAT',
 'province_KALIMANTAN SELATAN',
 'province_JAMBI',
 'province_NANGGROE ACEH DARUSSALAM',
 'province_KALIMANTAN TENGAH',
 'province_NUSA TENGGARA BARAT',
 'province_SULAWESI TENGAH',
 'province_BENGKULU',
 'province_NUSA TENGGARA TIMUR',
 'province_KEPULAUAN BANGKA BELITUNG',
 'province_SULAWESI 

In [8]:
all_var = load_data_from_pickle(file_path,'all_var.pkl')
# all_var['pre_tag'] = all_var.apply(lambda x: 1 if (x['education_REGULAR_COLLEGE_COURSE'] <0.5) & (x['B_7d']>2) else 0, axis = 1)
all_var['pre_tag'] = all_var.apply(lambda x: 1 if (x['industry_involved_TRADING_INDUSTRY'] >0) | (x['industry_involved_TRANSPORTASI_LOGISTIK']>0) else 0, axis = 1)

all_var.shape
all_var.head()

v2_score = load_data_from_pickle(file_path,'v2_score.pkl')
v2_score.head()

(41107, 277)

,loan_id,effective_date,due_date,approved_period,late_day,dpd1,age,monthly_salary,device_approval_ANDROID,device_approval_IOS,...,C_3d,C_7d,C_14d,C_21d,C_30d,C_60d,C_90d,C_180d,C_360d,pre_tag
0,443078651486502912,2020-03-07,2020-04-04,29,3.0,1,51,8500000.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,443080428537413632,2020-03-07,2020-03-28,22,-1.0,0,24,0.0,1,0,...,0.0,0.0,0.0,0.0,0.0,5.0,5.0,5.0,5.0,1
2,443085055811223552,2020-03-07,2020-04-04,29,-1.0,0,24,4000000.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,443086227296919552,2020-03-07,2020-03-21,15,17.0,1,32,5000000.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,443092329594003456,2020-03-07,2020-03-28,22,10.0,1,30,2000000.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


,loan_id,oldusermodelv2
0,437618487241121792,None
1,437654581873836032,None
2,437872384908296192,None
3,438009474178260992,None
4,438369463774380032,None


In [9]:
df_a = all_var[['loan_id','effective_date', 'due_date',	'approved_period','dpd1' ,'industry_involved_TRADING_INDUSTRY','industry_involved_TRANSPORTASI_LOGISTIK','pre_tag']].merge(v2_score,on='loan_id',how = 'left')
v2_bin = [ -np.inf, 628.0,
 638.0,
 644.0,
 650.0,
 654.0,
 659.0,
 664.0,
 670.0,
 678.0,np.inf]
df_a['oldusermodelv2'] = df_a['oldusermodelv2'].astype(float)
df_a['score_group'] = pd.cut(df_a['oldusermodelv2'],bins = v2_bin,duplicates='drop')
df_a.to_excel(os.path.join(file_path,'temp.xlsx'))
df_a.head()
df_a.shape

,loan_id,effective_date,due_date,approved_period,dpd1,industry_involved_TRADING_INDUSTRY,industry_involved_TRANSPORTASI_LOGISTIK,pre_tag,oldusermodelv2,score_group
0,443078651486502912,2020-03-07,2020-04-04,29,1,0,0,0,632.0,"(628.0, 638.0]"
1,443080428537413632,2020-03-07,2020-03-28,22,0,1,0,1,647.0,"(644.0, 650.0]"
2,443085055811223552,2020-03-07,2020-04-04,29,0,0,0,0,662.0,"(659.0, 664.0]"
3,443086227296919552,2020-03-07,2020-03-21,15,1,0,0,0,NaN,NaN
4,443092329594003456,2020-03-07,2020-03-28,22,1,1,0,1,645.0,"(644.0, 650.0]"


(41107, 10)